# Задание 3.
Проанализировать мощность АB-теста, проведеного с 2022-01-28 по 2022-02-03, группами 0 и 1, метрикой CTR.
## Задача:
1. При помощи библиотеки метода TTestIndPower оценить какой размер выборки нам потребуется при условии:
    * Использовать t-test для сравнения CTR по юзерам.
    * Размер эффекта равен 0.3
    * Мощность теста равнялась 0.8
2. Рассчитать чему оказался равен размер эффекта в нашем проведенном АB-тесте
3. Сделайте вывод, могли бы использовать меньший размер выборки при сохранении мощности на уровне 0.8.

### 1. Подготовка к работе

In [2]:
# Подгружаем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandahouse
from scipy import stats
from statsmodels.stats.power import TTestIndPower

### 2. Загрузка данных

In [3]:
# Подключение к БД
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': '*****************'
}

# Запрос к БД
query = '''
SELECT user_id,
       exp_group,
       countIf(user_id, action = 'like' ) as like,
       countIf(user_id, action = 'view' ) as view,
       like/view as ctr
FROM simulator_20220120.feed_actions
WHERE (toDate(time) >= '2022-01-28' AND toDate(time) <= '2022-02-03' 
AND exp_group in [0,1])
GROUP BY user_id, exp_group
'''

# Создание датафрейма
df = pandahouse.read_clickhouse(query, connection=connection)

In [10]:
# Проверка df
df

,user_id,exp_group,like,view,ctr
0,26117,1,5,21,0.238095
1,138232,1,5,29,0.172414
2,49204,1,10,62,0.161290
3,49286,1,7,19,0.368421
4,145957,1,54,129,0.418605
...,...,...,...,...,...
25994,3681,0,6,28,0.214286
25995,110956,1,12,63,0.190476
25996,132567,1,18,66,0.272727
25997,30872,1,25,82,0.304878


### 3. Анализ данных

Соберем необходимую информацию по группам 0 и 1

In [86]:
# Информация по группе 1
n_0 = df[df.exp_group == 0].shape[0]
std_0 = np.std(df[df.exp_group == 0].ctr)
mean_0 = np.mean(df[df.exp_group == 0].ctr)
print('n_0:', n_0,'std_0:', std_0, 'mean_0:', mean_0)

n_0: 12997 std_0: 0.08786700247435748 mean_0: 0.22195492956578472


In [87]:
# Информация по группе 1
n_1 = df[df.exp_group == 1].shape[0]
std_1 = np.std(df[df.exp_group == 1].ctr)
mean_1 = np.mean(df[df.exp_group == 1].ctr)
print('n_1:', n_1,'std_1:', std_1, 'mean_1:', mean_1)

n_1: 13002 std_1: 0.0901429791358773 mean_1: 0.23648646948093363


In [89]:
ratio = n_1 / n_0
ratio

1.0003847041624991

**Задача 1. При помощи библиотеки метода TTestIndPower оценить какой размер выборки нам потребуется, при условии:**
* размер эффекта = 0.3,
* мощность эффекта = 0.8
* для сравнения используем t-test 

In [71]:
# Необходимый размер выборки
sample_size = TTestIndPower().solve_power(effect_size = 0.3, power = 0.8, alpha = 0.05, ratio=ratio)
sample_size

175.35094960543293

**Вывод:** минимальный размер выборки равен 176 при условии, что размер эффекта равен 0.3, мощность нашего теста равна 0.8

**Задача 2. Рассчитать чему оказался равен размер эффекта в нашем проведенном АB-тесте**

In [65]:
# Рассчитаем меру эффекта (стандартизированная средняя разница)
effect_size_AB = (mean_1 - mean_0) / ( ((n_1 - 1) * std_1**2 + (n_0 - 1) * std_0**2) / (n_1 + n_0 - 2) )**0.5
effect_size_AB

0.1632528293594707

**Вывод:** размер эффекта оказался равен 0.16

**Задача 3. Сделайте вывод, могли бы использовать меньший размер выборки при сохранении мощности на уровне 0.8**

In [96]:
# Размер выборки нашего A/B теста
sample_size_AB = TTestIndPower().solve_power(effect_size = effect_size_AB, power = 0.8, alpha = 0.05, ratio=ratio)
sample_size_AB

589.8483073834748

**Вывод:** размер выборки в нашем тесте оказался равен 590. Таким образом при сохранинии мощности теста равным 0.8, мы могли бы использовать меньший размер выборки